In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, regularizers
from keras.applications import EfficientNetB0
from keras.applications.efficientnet import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score,roc_curve, auc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import os
from PIL import Image


cub_root = r'C:\Users\hiero\OneDrive\Documents\SOC-25_Intro-to-Deep-Learning\CUB_200_2011\CUB_200_2011'  # dataset root folder
images_txt = os.path.join(cub_root, 'images.txt')
bbox_txt = os.path.join(cub_root, 'bounding_boxes.txt')
images_dir = os.path.join(cub_root, 'images')

image_id_to_file = {}
with open(images_txt, 'r') as f:
    for line in f:
        img_id, file_path = line.strip().split()
        image_id_to_file[int(img_id)] = file_path

bbox_dict = {}
with open(bbox_txt, 'r') as f:
    for line in f:
        parts = line.strip().split()
        img_id, x, y, w, h = int(parts[0]), float(parts[1]), float(parts[2]), float(parts[3]), float(parts[4])
        bbox_dict[img_id] = (x, y, w, h)

m = len(image_id_to_file)  
images = np.zeros((m, 256, 256, 3), dtype=np.uint8)

for idx, img_id in enumerate(sorted(image_id_to_file.keys())):
    if idx >= m:
        break
    img_path = os.path.join(images_dir, image_id_to_file[img_id])
    image = Image.open(img_path).convert('RGB')
    x, y, w, h = bbox_dict[img_id]
    cropped = image.crop((x, y, x + w, y + h))
    cropped = cropped.resize((256, 256), Image.LANCZOS)
    images[idx] = np.array(cropped)

images = images.astype('float32')
images=images/255.0

In [ ]:
labels_txt = os.path.join(cub_root, 'image_class_labels.txt')

label_dict = {}
with open(labels_txt, 'r') as f:
    for line in f:
        img_id, class_id = line.strip().split()
        label_dict[int(img_id)] = int(class_id)  # class_id is 1-200

labels = np.zeros(m, dtype=np.int32)
for idx, img_id in enumerate(sorted(image_id_to_file.keys())):
    if idx >= m:
        break
    labels[idx] = label_dict[img_id]

labels=labels-1  # Convert to 0-199

In [ ]:
split_txt = os.path.join(cub_root, 'train_test_split.txt')
df = pd.read_csv(split_txt, sep=" ", header=None, names=['img_id', 'is_train'])
train_ids = df[df['is_train'] == 1]['img_id'].values
test_ids = df[df['is_train'] == 0]['img_id'].values
train_idx = [img_id - 1 for img_id in train_ids]
test_idx = [img_id - 1 for img_id in test_ids]
train_images = images[train_idx]
train_label = labels[train_idx]
test_images = images[test_idx]
test_label = labels[test_idx]

X_train, X_val, y_train, y_val = train_test_split(
    train_images, train_label, test_size=0.1, random_state=42, stratify=train_label
)

In [ ]:
# Define the base model
base_model = EfficientNetB0(
    input_shape=(256, 256, 3),
    include_top=False, 
    weights='imagenet'
)

base_model.trainable = False

#Create the new model on top
num_classes = 200
inputs = tf.keras.Input(shape=(256, 256, 3))

x = base_model(inputs, training=False) # Use the base model in inference mode
x = keras.layers.BatchNormalization()(x)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(num_classes, activation='softmax')(x)
model = keras.Model(inputs, outputs)

model.summary()

In [ ]:
# Compile the model for the first phase
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_val, y_val)
)

In [ ]:
base_model.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), # Very low learning rate
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

fine_tune_epochs = 10
total_epochs = 10 + fine_tune_epochs

history_fine_tune = model.fit(
    X_train, y_train,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    validation_data=(X_val, y_val)
)

predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)
accuracy = np.mean(predicted_classes == test_label)
print(f'Test accuracy: {accuracy:.4f}')

In [ ]:
def per_class_auc(y_true, y_pred, classes):
    y_true_bin = label_binarize(y_true, classes=classes)
    aucs = np.array([roc_auc_score(y_true_bin[:, i], y_pred[:, i]) for i in range(len(classes))])
    return np.round(aucs,3)

In [ ]:
from itertools import cycle

def plot_roc_curves(y_true, y_score, classes, title):
    y_true_bin = label_binarize(y_true, classes=classes)
    plt.figure(figsize=(6,4))
    for i, color in zip(range(len(classes)), cycle(['aqua','darkorange','cornflowerblue','green'])):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_score[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, color=color, lw=2, label=f'Class {classes[i]} (AUC = {roc_auc:.3f})')
    plt.plot([0,1], [0,1], 'k--', lw=1)
    plt.xlim([0.0, 1.0]); plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
aucs=np.array(per_class_auc(test_label,predictions,classes=[0,1,2,3]))
print(f"Neural Network AUCs: {aucs}")
plot_roc_curves(test_label,predictions,classes=[0,1,2,3],title="CNN ROC- Test")

In [ ]:

sample_img = test_images[20].astype('float32')
resized = tf.image.resize(sample_img[None, ...], (224, 224))
resized = preprocess_input(resized)
pred = model.predict(resized, verbose=0).argmax(axis=-1)[0]

plt.imshow(test_images[20].astype('uint8'))
plt.axis('off')
print('True label:', test_label[20], 'Predicted:', pred)